# Exploratory Data Analysis of GOES - GLM/ABI and ERA5 Data
Previous work has enabled me to retrieve and process large volumes of climate data from the three following sources: GOES-GLM, GOES-ABI, and ERA5 environmental variables on single levels and pressure levels. In this notebook, I investigate the quality of this data at the two primary stages of processing, temporal agreggation and spatial downsampling. To do this, I will produce visuals and check for missing data. Next, I will do correlational analysis between the lightning data and the environmental variables. Afterwards, I will check cross correlations between environmental variables. Then, I will investigate the distribution of my data using histograms, heatmaps, and time series analysis.

We will begin by loading in 1 months worth of data using the retrieve GLM function in data retriever 2.

In [1]:
# imports
from data_retreiver_2 import retrieve_goes_glmf, parallel_interp
import xarray as xr
import cdsapi
import netCDF4 as nc
from goes2go import GOES
import requests
from datetime import datetime, timedelta
import os
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np
from numba import njit, prange
from scipy.interpolate import griddata
from scipy.spatial import cKDTree
import multiprocessing
import time
import dask
from dask import delayed, compute



In [6]:
import os, glob
for file in glob.glob("OR_*"):
    os.remove(file)
    print(f"Deleted: {file}")


Deleted: OR_GLM-L2-GLMF-M3_G16_e20200425184000.nc
Deleted: OR_GLM-L2-GLMF-M3_G16_e20200425181000.nc
Deleted: OR_GLM-L2-GLMF-M3_G16_e20200425182500.nc
Deleted: OR_GLM-L2-GLMF-M3_G16_e20200425180500.nc
Deleted: OR_GLM-L2-GLMF-M3_G16_e20200425183000.nc
Deleted: OR_GLM-L2-GLMF-M3_G16_e20200425182000.nc
Deleted: OR_GLM-L2-GLMF-M3_G16_e20200425183500.nc
Deleted: OR_GLM-L2-GLMF-M3_G16_e20200425181500.nc


In [3]:
start_date = datetime(2020, 4, 1)
end_date = datetime(2020, 4, 30)
daily_files = []

def process_day(date):
    """Processes a single day's data."""
    output_file = f"{date.strftime('%B%Y')}/FED_Window_{date.strftime('%Y%m%d')}.nc"
    print(f"Processing: {date.strftime('%Y-%m-%d')}")
    retrieve_goes_glmf(date, output_file)  # Process daily data
    return output_file

for i in range((end_date - start_date).days + 1):
    daily_files.append(process_day(start_date + timedelta(days=i)))

print("Finished processing all days!")



Processing: 2020-04-01


Processing files for year 2020, day 92, hour 1Processing files for year 2020, day 92, hour 0Processing files for year 2020, day 92, hour 3Processing files for year 2020, day 92, hour 2



Processing files for year 2020, day 92, hour 4
Processing files for year 2020, day 92, hour 5
Processing files for year 2020, day 92, hour 6
Processing files for year 2020, day 92, hour 7
Processing files for year 2020, day 92, hour 8
Processing files for year 2020, day 92, hour 9
Processing files for year 2020, day 92, hour 10
Processing files for year 2020, day 92, hour 11
Processing files for year 2020, day 92, hour 12
Processing files for year 2020, day 92, hour 13
Processing files for year 2020, day 92, hour 14
Processing files for year 2020, day 92, hour 15
Processing files for year 2020, day 92, hour 16
Processing files for year 2020, day 92, hour 17
Processing files for year 2020, day 92, hour 18
Processing files for year 2020, day 92, hour 19
Processing files for year 2020, day 92, hour 20
Pr

/gpfs/fs1/home/o/oneill/mamous3/lightning/data_collection/data_retreiver_2.py:325: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  combined_fed.to_netcdf(output_file)
/gpfs/fs1/home/o/oneill/mamous3/lightning/data_collection/data_retreiver_2.py:325: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  combined_fed.to_netcdf(output_file)


Saved 24-hour FED data to April2020/FED_Window_20200401.nc
Faulty links have been saved to faulty_links.txt
Processing: 2020-04-02
Processing files for year 2020, day 93, hour 0Processing files for year 2020, day 93, hour 1Processing files for year 2020, day 93, hour 2Processing files for year 2020, day 93, hour 3



Processing files for year 2020, day 93, hour 4
Processing files for year 2020, day 93, hour 5
Processing files for year 2020, day 93, hour 6
Processing files for year 2020, day 93, hour 7
Processing files for year 2020, day 93, hour 8
Processing files for year 2020, day 93, hour 9
Processing files for year 2020, day 93, hour 10
Processing files for year 2020, day 93, hour 11
Processing files for year 2020, day 93, hour 12
Processing files for year 2020, day 93, hour 13
Processing files for year 2020, day 93, hour 14
Processing files for year 2020, day 93, hour 15
Processing files for year 2020, day 93, hour 16
Processing files for year 2020, day 93, hour 17
Processing file

/gpfs/fs1/home/o/oneill/mamous3/lightning/data_collection/data_retreiver_2.py:325: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  combined_fed.to_netcdf(output_file)
/gpfs/fs1/home/o/oneill/mamous3/lightning/data_collection/data_retreiver_2.py:325: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  combined_fed.to_netcdf(output_file)


...

Saved 24-hour FED data to April2020/FED_Window_20200428.nc
Faulty links have been saved to faulty_links.txt
Processing: 2020-04-29
Processing files for year 2020, day 120, hour 2Processing files for year 2020, day 120, hour 0Processing files for year 2020, day 120, hour 3Processing files for year 2020, day 120, hour 1



Processing files for year 2020, day 120, hour 4
Processing files for year 2020, day 120, hour 5
Processing files for year 2020, day 120, hour 6
Processing files for year 2020, day 120, hour 7
Processing files for year 2020, day 120, hour 8
Processing files for year 2020, day 120, hour 9
Processing files for year 2020, day 120, hour 10
Processing files for year 2020, day 120, hour 11
Processing files for year 2020, day 120, hour 12
Processing files for year 2020, day 120, hour 13
Processing files for year 2020, day 120, hour 14
Processing files for year 2020, day 120, hour 15
Processing files for year 2020, day 120, hour 16
Processing files for year 2020, day 120, hour 

/gpfs/fs1/home/o/oneill/mamous3/lightning/data_collection/data_retreiver_2.py:325: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  combined_fed.to_netcdf(output_file)
/gpfs/fs1/home/o/oneill/mamous3/lightning/data_collection/data_retreiver_2.py:325: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  combined_fed.to_netcdf(output_file)


Saved 24-hour FED data to April2020/FED_Window_20200429.nc
Faulty links have been saved to faulty_links.txt
Processing: 2020-04-30
Processing files for year 2020, day 121, hour 0Processing files for year 2020, day 121, hour 3Processing files for year 2020, day 121, hour 1Processing files for year 2020, day 121, hour 2



Processing files for year 2020, day 121, hour 4
Processing files for year 2020, day 121, hour 5
Processing files for year 2020, day 121, hour 6
Processing files for year 2020, day 121, hour 7
Processing files for year 2020, day 121, hour 8
Processing files for year 2020, day 121, hour 9
Processing files for year 2020, day 121, hour 10
Processing files for year 2020, day 121, hour 11
Processing files for year 2020, day 121, hour 12
Processing files for year 2020, day 121, hour 13
Processing files for year 2020, day 121, hour 14
Processing files for year 2020, day 121, hour 15
Processing files for year 2020, day 121, hour 16
Processing files for year 2020, day 121, hour 

/gpfs/fs1/home/o/oneill/mamous3/lightning/data_collection/data_retreiver_2.py:325: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  combined_fed.to_netcdf(output_file)
/gpfs/fs1/home/o/oneill/mamous3/lightning/data_collection/data_retreiver_2.py:325: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  combined_fed.to_netcdf(output_file)


Saved 24-hour FED data to April2020/FED_Window_20200430.nc
Faulty links have been saved to faulty_links.txt
Finished processing all days!


After loading the data, we will check the quality. First, ensure there are no missing days or hours in the dataset. Then, add latitude and longitude to the data by using calc lat lon. Finally, slice the dataset into 4 regions, SW, NW, SE, and NE CONUS. 

In [ ]:
from data_retreiver_2 import calc_latlon, get_xy_from_latlon, parallel_interp

april_fed = []
for file in daily_files:
    ds = xr.open_dataset(daily_files[0])
    ds = calc_latlon(ds)
    ds.sortby('time')
    april_fed.append(ds)
    ds.close

april_fed_total = xr.concat(april_fed,'time')

Next, create three different visuals for each region. First, do a 24 hr animation of the day with the highest FED. Then, do a 30 day animation with a heat map of lightning for each day. Finally, make a heat map for the total month.

Save these figures for later.

At the moment, we have temporally smoothed the data, but now we want to spatially smooth the data in order to make meaningful comparisons with lower resolution datasets (ERA5). 

Run parallel_interp on each day of the data. Try out parallelism for this as well, keep worker numbers low though.

Repeat the visualization process, and compare to previous figures. 

We have now visually confirmed the quality of the data.


Now, we will introduce the environmental variables to our EDA. Use retrieve ABI to download ABI data variables of interest for the month. You can use parallel processing again for this, 10-16 workers max. Verify the data in the same way we verified the lightning data.

Getting ERA5 Data is simple and relatively fast, so use the cdsapi and retrieve it for this month.

We can do some simple checks for the ERA5 data, but it is likely that there are no direct issues as the ECMFW is a trusted organization responsible for this reanalysis data.

Now we will begin correlation analysis. How do we do this? By using xarray’s .corr which measures the linear pearson correlation coefficient for datasets across a specified dimension (i. e. time, space, etc)
Note: Make sure to cite parameterizations.


Perform correlational analysis spatially, temporally, and spatiotemporally

* Lightning vs Individual Controls
* Individual Control vs Individual Controls
* Lightning vs Parameterizations 
* Parameterizations vs Individual Controls
* Parameterizations vs Parameterizations